In [1]:
import numpy as np
import pandas as pd
import time
from sklearn.svm import SVC,LinearSVC
from sklearn.multiclass import OneVsRestClassifier

########READ in Data########
readdata = pd.read_csv("/Users/academic/Desktop/TheGrandTour/data/wineData.txt", sep="\t", header=None);
data = np.array(readdata);
data = np.delete(data, 0, 0)
data = np.swapaxes(data,0,1)


# Need to seperate the classification dimension:
classIndex = np.shape(data)[0] - 1
classification = data[classIndex]
data = np.delete(data, classIndex, axis=0)
data = data.astype(float)


# Make list of colours for each number:
colour = np.chararray((len(classification)),unicode = True)
for i in range(len(classification)):
    if classification[i] == '1':
        colour[i] = 'r'
    elif classification[i] == '2':
        colour[i] = 'b'
    elif classification[i] == '3':
        colour[i] = 'g'


# Normalise data
for i in range(0, np.shape(data)[0]):
    data[i,:] = data[i,:] - np.min(data[i,:])
    data[i,:] = (data[i,:]/np.ndarray.max(data[i,:]))*2 - 1

nPoints = np.shape(data)[1]
n = np.shape(data)[0]
d = n


def step(coord,n,stepSize):
    coord[n] = coord[n] + stepSize
    return coord

def getPolars(coord):
    '''polars = np.zeros((2,d))
    polars[1,0] = np.pi/2
    j = 1
    for i in range(d-1):
        polars[1,j] = coord[i]
        j+=1
    j = 0
    for i in range(d-1,2*d - 1):
        polars[0,j] = coord[i]
        j += 1
    return polars '''
    polars = np.zeros((2,d-1))
   
    for i in range(d-1):
        polars[0,i] = coord[i]
    j = 1
    polars[1,0] = np.pi/2
    for i in range(d-1,2*d - 3):
        polars[1,j] = coord[i]
        j += 1
    return polars 

def RotationMatrix(i, j, d, theta):
    R = np.identity(d)
    R[i,i] = np.cos(theta)
    R[i,j] = -1*np.sin(theta)
    R[j,i] = np.sin(theta)
    R[j,j] = np.cos(theta)
    return R

def getWigner(polar):
    wig = RotationMatrix(0,1,d,polar[0])
    for i in range(1,d-1):
        wig = np.matmul(RotationMatrix(i,i+1,d,polar[i]),wig) 
    return wig
    
def getX(coord):
    polars = getPolars(coord)
    X = np.ones((2,d))
    X[0] = X[0]*0
    X[0,0] = 1
    i = 1
    for j in range(0,d):
        k = j 
        if j != d-1:
            X[i,j] = np.cos(polars[i,j])
        else:
            k = j-1
            X[i,j] = np.sin(polars[i,j-1])
        while k > 0:
            k -= 1
            X[i,j] = X[i,j]*np.sin(polars[i,k])
    wig = getWigner(polars[0])
    X[0] = wig.dot(X[0])
    X[1] = wig.dot(X[1])
    return X

def Transform(data, coord):
    nPoints = np.shape(data)[1]
    a = np.zeros((2,nPoints))
    X = getX(coord)  
    for p in range(0,nPoints):
        a[0,p] = np.dot(X[0],data[:,p])
        a[1,p] = np.dot(X[1],data[:,p])
    return a

def SVM(a):   
    xData = a[0]
    yData = a[1]
    nData = np.vstack((xData, yData)).T
    clf = OneVsRestClassifier(SVC(kernel='linear'), n_jobs=-1)
    clf.fit(nData, classification)
    score = clf.score(nData, classification)
    return score

def function(coords):
    a = Transform(data,coords)
    w = SVM(a)
    return w

In [2]:
import matplotlib.pyplot as plt
from scipy.optimize import fmin
import numpy.random as rnd
#import corner
# New np.random seed each iteration
#rnd.seed(1000)
#Can you place a bound on the the number of dimensions for a curve fitting based on the sd of the parameters mcmc?
rnd.seed(1)

def Proposal_function(beta0, step_sigma):

    return (beta0 + rnd.randn(len(beta0))*step_sigma)%2*np.pi




def Metropolis_Hastings(function, beta_dimensions,betamax = np.pi*2,betamin = 0):
    # Parameters for algorithm:
    rnd.seed(int(time.time()))
    step_sigma = 0.3*np.pi
    step_max = 10000
    print(step_max)
    # Initialise arrays to store data
    betalist = np.zeros((step_max,beta_dimensions))
    likelyhoodlist = np.zeros(step_max)
    accepts = 0
    rejects = 0

    # Initial conditions

    betalist[0] = (rnd.rand(beta_dimensions)*(betamax-betamin)+betamin)
    likelyhood_0 = function(betalist[0])
    likelyhoodlist[0] = likelyhood_0
    # Start sampling
    for i in range(1, step_max):
        betacandidate = Proposal_function(betalist[i-1], step_sigma)
        likelyhood_1 = function(betacandidate)
        if (likelyhood_1 / likelyhood_0) > rnd.uniform(0, 1):
            betalist[i] = betacandidate
            print(i, " ", betacandidate, end="\r")
            likelyhoodlist[i] = likelyhood_1
            likelyhood_0 = likelyhood_1
            accepts = accepts + 1

        else:
            betalist[i] = betalist[i-1]
            likelyhoodlist[i] = likelyhoodlist[i-1]
            rejects = rejects + 1
        if (i%59):
            step_sigma = step_sigma*(1-(0.234-(accepts/(accepts+rejects))))
    print(accepts)
    print(betalist[:][1])
    print(rejects)
    print("acceptance rate = " + str(accepts/(rejects+accepts)))
    print("Lists created. Plotting results...")
    np.save("tempbetalist3",betalist)
    fig1, axarr = plt.subplots(1, 2, sharey=True)

    axarr[0].hist(betalist[-round(step_max/4):][0:2],bins=1000,orientation="horizontal",histtype="stepfilled")
    labels = ["Beta["+str(beta_dimensions-1-i)+ "]" for i in range(beta_dimensions)]
    axarr[0].legend(labels)

    axarr[1].plot(np.arange(len(betalist)),betalist[:][0:2], linewidth = 0.1)
    axarr[0].set_xlabel("Value of coefficient")
    axarr[0].set_ylabel("Acceptances of coefficient")
    axarr[1].set_xlabel("Iteration")
    # Plot model fitting

    # Print median betas with sd to console.
    coeffs = np.median(betalist[int(len(betalist)/4):],axis=0)
    stddev = np.std(betalist[int(len(betalist)/4):],axis=0)
    for i in range (beta_dimensions):
        print("Median beta[" + str(i) + "] = " + str(coeffs[i]) + ", standard deviation = " + str(stddev[i]) + ".")
    return coeffs, betalist




dim = 2*n - 3

coeffs, betalist = Metropolis_Hastings(function, dim,betamax = np.pi*2,betamin = 0)
plt.show()

10000
1   [3.51676349 2.90138237 0.04370362 6.23166815 5.4590727  2.66260894
 2.57326035 0.11172799 2.98681518 4.85973014 1.3893848  4.06372736
 3.37221045 1.34020047 5.43665268 4.71270193 2.95506239 0.79749748
2   [2.1957766  0.60965748 0.2971077  2.51463154 3.8650328  2.34157719
 5.23970466 3.81872963 1.17395733 1.26882966 1.24035237 3.33161309
 2.01620282 2.58152421 0.90210042 5.74971385 2.43922256 6.25661764
3   [4.55406701 4.32129648 1.07928548 3.21313253 1.65296286 2.41595988
 2.21927806 4.89712863 6.28263014 3.59248133 3.89195497 4.85220557
 4.67675664 0.26180794 1.46379005 2.12627025 1.48186843 5.06208403
4   [3.84706826 3.3207479  2.12219354 1.79465804 0.47299627 1.63478292
 4.35822367 4.88079302 5.93185808 5.33855365 2.89474022 4.0168899
 1.5135913  2.74321622 3.37680195 0.70945368 3.55500592 4.8012596
5   [0.14445936 0.22554294 1.39758146 2.46085095 1.17506967 4.18928032
 5.76292157 1.87315164 3.81458708 2.70587003 4.28218144 1.26432393
 0.77394144 2.04197677 4.96144275 0.33

43   [5.74851211 2.13553233 6.2216759  4.6362371  3.82025931 3.11153172
 2.74300333 4.43663197 1.90457946 1.65465145 4.65852663 1.49009755
 2.29865223 3.52326625 0.28784985 4.55980433 1.17331329 2.85685232
44   [5.66527867 5.20599823 5.1657612  0.69331363 2.58186344 5.2952897
 5.29878909 4.60939056 3.86257561 2.94270995 5.34643837 5.9461889
 4.10747324 0.26542062 5.95596803 3.37732685 0.23678432 6.23167112
45   [2.5975388  1.62478526 4.76833134 2.4174938  5.1336225  3.98815381
 5.67174191 0.33549238 5.52800986 5.85097424 4.82436359 2.5463362
 2.0931048  2.57173078 0.01581918 6.00325778 4.14951391 1.64219534
46   [0.51577108 0.5181859  1.82002027 4.40555687 5.50030383 0.33527068
 4.21298236 5.06812871 5.14907017 3.63981284 1.12012955 2.05625331
 0.86725042 2.11912255 1.00715426 5.95294801 2.32834315 4.70704402
47   [0.53427772 5.44102985 1.80444677 3.62738519 0.95516669 3.24795565
 5.330008   3.4265955  4.7467118  3.97127253 4.09083914 4.29466684
 2.1611752  1.2392887  2.04618656 4.9627

/Users/karanmukhi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: overflow encountered in multiply
  if sys.path[0] == '':


JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/Users/karanmukhi/anaconda3/lib/python3.6/runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...b/python3.6/site-packages/ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
/Users/karanmukhi/anacon

In [4]:
i

12